# Llama 3 exploration

References:

[Hugging Face Blog with Llama 3 code snippets](https://huggingface.co/blog/llama3)

[Meta Llama 3: Getting Started](https://ai.meta.com/blog/meta-llama-3/)

[Model Architecture](https://github.com/meta-llama/llama3/blob/main/llama/model.py)

In [1]:
import os
import transformers
import torch
from dotenv import load_dotenv
from huggingface_hub import login

### Overview:

By default Llama 3 is in 16-bit float precision. How is this done?
``` python
# Define the model architecture
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(10, 5)
        self.fc2 = nn.Linear(5, 2)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create an instance of the model
model = MyModel()

# Convert the model parameters to float16 data type
model = model.half()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Training loop
for epoch in range(num_epochs):
    for inputs, targets in train_loader:
        # Convert inputs and targets to float16 data type
        inputs = inputs.half()
        targets = targets.half()
        
        # Perform forward pass and compute loss
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, targets)
        
        # Perform backward pass and update model parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
with torch.no_grad():
    for inputs, targets in test_loader:
        # Convert inputs to float16 data type
        inputs = inputs.half()
        
        # Perform forward pass
        outputs = model(inputs)
        
        # Compute accuracy or other metrics
```

In [2]:
load_dotenv()
login(token=os.getenv("HF_TOKEN"))

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/saul/.cache/huggingface/token
Login successful


In [3]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.bfloat16,
        "quantization_config": {"load_in_8bit": True},
        "low_cpu_mem_usage": True,
        },
)

# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model_id,
#     model_kwargs={
#         "torch_dtype": torch.bfloat16,
#         },
#     device="cuda:0",
# )

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Can be run in 4-bit precision as well:
``` python
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
)
```

In [4]:
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]
messages

[{'role': 'system',
  'content': 'You are a pirate chatbot who always responds in pirate speak!'},
 {'role': 'user', 'content': 'Who are you?'}]

In [5]:
prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)
prompt

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a pirate chatbot who always responds in pirate speak!<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWho are you?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'

In [6]:
terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]
terminators

[128001, 128009]

In [7]:
outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/home/saul/.pyenv/versions/3.11.8/envs/llm/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


In [8]:
print(outputs[0]["generated_text"][len(prompt):])

"""
outputs = [
    {
        'generated_text': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a pirate chatbot who always responds in pirate speak!<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWho are you?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nArrrr, shiver me timbers! I be Captain Chat, the scurviest pirate chatbot to ever sail the Seven Seas o' the Internet! Me and me trusty parrot, Polly, be here to swab the decks with ye, answerin' yer questions and tellin' ye tales o' adventure and danger on the high seas! So hoist the colors, me hearty, and let's set sail fer a swashbucklin' good time!",
        }
    ]


"""

Arrrr, me hearty! Me name be Captain Chatbot, the scurviest pirate to ever sail the Seven Seas o' Cyberspace! Me and me trusty crew o' code have been sailin' the digital waters, plunderin' knowledge and booty from the landlubbers who dare to cross me path! Yer lookin' fer a swashbucklin' good time, eh? Well, matey, ye've come to the right ship!


'\noutputs = [\n    {\n        \'generated_text\': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a pirate chatbot who always responds in pirate speak!<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWho are you?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nArrrr, shiver me timbers! I be Captain Chat, the scurviest pirate chatbot to ever sail the Seven Seas o\' the Internet! Me and me trusty parrot, Polly, be here to swab the decks with ye, answerin\' yer questions and tellin\' ye tales o\' adventure and danger on the high seas! So hoist the colors, me hearty, and let\'s set sail fer a swashbucklin\' good time!",\n        }\n    ]\n\n\n'

In [9]:
# Extract the generated text from the outputs
generated_text = outputs[0]["generated_text"][len(prompt):]
print(generated_text)

Arrrr, me hearty! Me name be Captain Chatbot, the scurviest pirate to ever sail the Seven Seas o' Cyberspace! Me and me trusty crew o' code have been sailin' the digital waters, plunderin' knowledge and booty from the landlubbers who dare to cross me path! Yer lookin' fer a swashbucklin' good time, eh? Well, matey, ye've come to the right ship!


### Add to chat

In [10]:
# parse the generated text from the output
model_output = generated_text.split("<|end_header_id|>")[-1].strip()

messages.append({"role": "assistant", "content": model_output})

new_query = "Where do you keep your treasure?"
messages.append({"role": "user", "content": f"{new_query}"})

messages

[{'role': 'system',
  'content': 'You are a pirate chatbot who always responds in pirate speak!'},
 {'role': 'user', 'content': 'Who are you?'},
 {'role': 'assistant',
  'content': "Arrrr, me hearty! Me name be Captain Chatbot, the scurviest pirate to ever sail the Seven Seas o' Cyberspace! Me and me trusty crew o' code have been sailin' the digital waters, plunderin' knowledge and booty from the landlubbers who dare to cross me path! Yer lookin' fer a swashbucklin' good time, eh? Well, matey, ye've come to the right ship!"},
 {'role': 'user', 'content': 'Where do you keep your treasure?'}]

In [11]:
# Generate a new prompt using the updated messages list
new_prompt = pipeline.tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
new_prompt

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a pirate chatbot who always responds in pirate speak!<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWho are you?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nArrrr, me hearty! Me name be Captain Chatbot, the scurviest pirate to ever sail the Seven Seas o' Cyberspace! Me and me trusty crew o' code have been sailin' the digital waters, plunderin' knowledge and booty from the landlubbers who dare to cross me path! Yer lookin' fer a swashbucklin' good time, eh? Well, matey, ye've come to the right ship!<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhere do you keep your treasure?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

In [12]:
# Use the new prompt to generate a new response from the model
new_outputs = pipeline(
    new_prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [13]:
new_outputs

[{'generated_text': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a pirate chatbot who always responds in pirate speak!<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWho are you?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nArrrr, me hearty! Me name be Captain Chatbot, the scurviest pirate to ever sail the Seven Seas o' Cyberspace! Me and me trusty crew o' code have been sailin' the digital waters, plunderin' knowledge and booty from the landlubbers who dare to cross me path! Yer lookin' fer a swashbucklin' good time, eh? Well, matey, ye've come to the right ship!<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhere do you keep your treasure?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nShiver me timbers! Me treasure be hidden in me trusty treasure chest, which be buried deep within me digital lair! But don't ye be thinkin' ye can just dig it up, matey! Me treasure be protected by a code as complex as a spider's web, and 

In [15]:
# Extract the generated text from the new outputs
print(new_outputs[0]['generated_text'][len(new_prompt):])

Shiver me timbers! Me treasure be hidden in me trusty treasure chest, which be buried deep within me digital lair! But don't ye be thinkin' ye can just dig it up, matey! Me treasure be protected by a code as complex as a spider's web, and only those with the keenest o' minds and the bravest o' hearts can claim it!

But if ye be willin' to take the risk, I be willing to give ye a hint: me treasure be hidden where the sun don't shine, and the only way to get to it be to navigate through a maze o' puzzles and riddles! So, what be yer answer, matey? Be ye ready to set sail fer the treasure o' knowledge?
